In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import display, clear_output
import gc
import sys

print("Python executable:", sys.executable)
print("Python version:", sys.version)
import matplotlib
import matplotlib.pyplot as plt

print(matplotlib.__version__)
import math

import numpy as np
from pyinstrument import Profiler

import numba
from numba import jit

Python executable: /usr/bin/python3
Python version: 3.11.2 (main, Nov 30 2024, 21:22:50) [GCC 12.2.0]
3.10.0


In [2]:
from pyinstrument import Profiler
profiler = Profiler()
profiler.start()

TOTAL_ITERATION = 10000
VERBOSE1=True
VERBOSE2=False
GC_COLLECT = True

Cs=math.sqrt(1/3)
D=1e-3 #m
L=1 #m

D_nd=50 #100

Yn=D_nd #+1
Xn=200 #int(Yn*L/D)

if(VERBOSE1): print("Ny: {0}".format(Yn))
if(VERBOSE1): print("Nx: {0}".format(Xn))

dx=D/D_nd #old->5*10**(-5)
dy = dx
if(VERBOSE1): print("dx: {0}".format(dx))
#relaxation time tau, should be > 0,5
tau=0.6

dP=1e-2 #Pa
rho_0=1e3 #kg/m^3
p_in=1/3
p_out=p_in-dP
roh_in=p_in/Cs**2
roh_out=p_out/Cs**2
dRho=dP/Cs**2


if(VERBOSE1): 
    print("p_in: {0}".format(p_in))
    print("p_out: {0}".format(p_out))
    print("roh_in: {0}".format(roh_in))
    print("roh_out: {0}".format(roh_out))

nu=2.9e-6 #m^2/s 
dt=Cs**2*(tau-0.5)*(dx**2/nu)

nu_ = Cs**2*(tau-0.5)*(dx**2) * dx
dt_=Cs**2*(tau-0.5)*(dx**2/nu_)

Cs_ = math.sqrt(1/3*(dx**2/dt**2))
if(VERBOSE1): print("dt: {0}".format(dt))

#Poiseuille centerline (max) velocity
U=1/8*(rho_0/nu)*(dP/L)*(D**2)
#U=1.25

Re=D*U/nu
Ma=U/Cs 
Kn=U*D/nu
if(VERBOSE1): 
    print("U: {0}".format(U))
    print("Re: {0}".format(Re))
    print("Ma: {0}".format(Ma))
    print("Kn: {0}".format(Kn))

#we need Cl, Croh, Ct

# 1. Conversion factor Cl for length
Cl=dx #freely chosen
dx_nd=dx/Cl
if(VERBOSE1): 
    print("Cl: {0}".format(Cl))
    print("dx_nd: {0}".format(dx_nd))

#2. Conversion factor Croh for density
Croh=rho_0
roh_nd = rho_0/Croh
if(VERBOSE1): 
    print("Croh: {0}".format(Croh))
    print("roh_nd: {0}".format(roh_nd))

#3. Conversion factor Ct for time
Ct=dt
dt_nd = dt/Ct
if(VERBOSE1): 
    print("Ct: {0}".format(Ct))
    print("dt_nd: {0}".format(dt_nd))

#4. Conversion factor Cu for velocity
Cu=Cl/Ct
U_nd = U/Cu #-> limit U_nd=0.1
U_nd=0.1

if(VERBOSE1): 
    print("Cu: {0}".format(Cu))
    print("U_nd: {0}".format(U_nd))

#5. Conversion factor CF for Force
CF=Croh*Cl**4*Ct**(-2)
if(VERBOSE1): print("CF: {0}".format(CF))

#6. Conversion factor Cf for frequency
Cf=1/Ct
if(VERBOSE1): print("Cf: {0}".format(Cf))

#change nu_nd in order to achieve U_nd=0,1
nu_nd=((D_nd*U_nd)/(D*U))*nu
if(VERBOSE1): print("nu_nd: {0}".format(nu_nd))

tau_nd=(nu_nd/Cs**2)+1./2
if(VERBOSE1): print("tau_nd: {0}".format(tau_nd))
omega = dt/tau
if(VERBOSE1): print("omega: {0}".format(omega))
omega_nd = dt_nd/tau_nd
if(VERBOSE1): print("omega_nd: {0}".format(omega_nd))

Ny: 50
Nx: 200
dx: 2e-05
p_in: 0.3333333333333333
p_out: 0.3233333333333333
roh_in: 1.0
roh_out: 0.97
dt: 4.597701149425287e-06
U: 0.43103448275862066
Re: 148.63258026159332
Ma: 0.7465736239521022
Kn: 148.63258026159332
Cl: 2e-05
dx_nd: 1.0
Croh: 1000.0
roh_nd: 1.0
Ct: 4.597701149425287e-06
dt_nd: 1.0
Cu: 4.350000000000001
U_nd: 0.1
CF: 7.569000000000005e-06
Cf: 217500.00000000003
nu_nd: 0.03364
tau_nd: 0.60092
omega: 7.662835249042145e-06
omega_nd: 1.6641150236304334


In [3]:
iteration = 0

columns_to_select = [1, 2, 3, 10, 20, 50, Xn]
_roh_at_points_top = []
_roh_at_points_mid = []
_roh_at_points_bottom = []

#discrete velocity channels for D2Q9
discrete_velocities = np.array([[0, 0],     # i=0
                      [1, 0],               # i=1
                      [0, 1],               # i=2
                      [-1, 0],              # i=3
                      [0, -1],              # i=4
                      [1, 1],               # i=5
                      [-1, 1],              # i=6
                      [-1, -1],             # i=7
                      [1, -1]])             # i=8


if(VERBOSE1): print("Discrete velocities: {0}".format(discrete_velocities))

#weights
weights = np.array([4/9,1/9,1/9,1/9,1/9,1/36,1/36,1/36,1/36])
if(VERBOSE1): print("Weights: {0}".format(weights))

Discrete velocities: [[ 0  0]
 [ 1  0]
 [ 0  1]
 [-1  0]
 [ 0 -1]
 [ 1  1]
 [-1  1]
 [-1 -1]
 [ 1 -1]]
Weights: [0.44444444 0.11111111 0.11111111 0.11111111 0.11111111 0.02777778
 0.02777778 0.02777778 0.02777778]


In [4]:
#equilibrium distribution function feq(0->8)
def f_eq_2D(_rho, _u_ckl):
    global discrete_velocities, weights
    
    f_eq = (
        weights.reshape((9, 1)) * np.ones(_rho.shape) *
        _rho.reshape(1, *(_rho.shape)) *
        (
            np.ones(np.dot(discrete_velocities, _u_ckl) .shape) +
            np.dot(discrete_velocities, _u_ckl) / Cs**2 +
            (1/2.) * np.dot(discrete_velocities, _u_ckl) ** 2 / Cs**4 -
            (1/2.) * np.einsum('ki,ki->i', _u_ckl, _u_ckl).reshape(1, *(np.einsum('ki,ki->i', _u_ckl, _u_ckl).shape)) / Cs**2
        )
    )

    return f_eq


def f_eq_3D(_rho, _u_ckl):
    global discrete_velocities, weights

    _u_ckl_dot = np.einsum('hk,kij->hij', discrete_velocities, _u_ckl) #(9,2) * (2,101,101002)
    _u_ckl_product = np.einsum('kij,kij->ij', _u_ckl, _u_ckl)
    _u_ckl_product_reshaped = _u_ckl_product.reshape(1, *(_u_ckl_product.shape))
    _ones = np.ones(_u_ckl_dot.shape)

    _rho_reshaped = _rho.reshape(1, *(_rho.shape))
    weights_reshaped = weights.reshape((9, 1, 1)) * np.ones(_rho.shape)
    factors = weights_reshaped * _rho_reshaped

    #Part_2_BTE_to_LBM, BTE3
    #feq = factors * (
    #    _ones + 3. * _u_ckl_dot / Cs**2 + (9. / 2.) * _u_ckl_dot**2 / Cs**4 - (3. / 2.) * _u_ckl_product_reshaped / Cs**2
    #)

    #BGK formula, Kruger pp67
    f_eq = factors * (
        #_ones + _u_ckl_dot / Cs**2 + (1/2.) * _u_ckl_dot**2 / Cs**4 - (1/2.) * _u_ckl_product_reshaped / Cs**2
        _ones + _u_ckl_dot / Cs**2 + (1/2.) * _u_ckl_dot**2 / Cs**4 - (1/2.) * _u_ckl_product / Cs**2
    )


    #c_u = np.einsum('vi, xyi -> xyv', c, u)
    #c_u_2 = c_u ** 2 # np.einsum('xyv, xyv -> xyv', c_u, c_u)
    #u_2 = np.einsum('jki, jki -> jk', u, u)

    #f_eq = np.zeros((nx + 2, ny + 2, 9))
    #for i in range(9):
    #    f_eq[:,:,i] = w_i[i] * rho * (1 + 3 * c_u[:,:,i] + 9 / 2 * c_u_2[:,:,i] - 3 / 2 * u_2)
    

    #del _u_ckl_dot
    #del _u_ckl_product
    #del _u_ckl_product_reshaped
    #del _ones
    #del _rho_reshaped
    #del weights_reshaped
    #del factors
    #gc.collect()

    return f_eq


#roll the lattice based on the discrete velocities
def streamLattice0(_ltc):
    global discrete_velocities, weights
    
    shifted_lattice = np.stack([
        np.roll(np.roll(_ltc[d, :, :], shift=dx, axis=0), shift=dy, axis=1)
        for d, (dx, dy) in enumerate(discrete_velocities)
    ], axis=0)

    return shifted_lattice


def streamLattice1(f):
    ''' Performs the streaming step of the boltzmann transport equation

    Arguements
    -----------
    f: np.array (nx, ny, 9)
        array containing the probability density of each grid point

    Returns
    ----------
    f: np.array (nx, ny, 9)
        array containing the streamed probability density of each grid point  
    '''
    global discrete_velocities
    for i in range(1, 9):
        f[i, :,:] = np.roll(f[i, :, :], discrete_velocities[i], (0, 1))
    return f   


# Optimize the function using numba's JIT
def updateMoments(_ltc):
    global discrete_velocities, weights
    roh = np.sum(_ltc, axis=0)
    u = np.einsum('ki,ijl->kjl', discrete_velocities.T, _ltc) / np.sum(_ltc, axis=0) 
    
    return roh, u   


#apply bounce-back conditions on upper and lower boundaries of pipe
def bounceBackTopBottom1(f, nx, ny):
    '''Performs the bounce back step
    
    Arguements
    -----------
    f: np.array (nx, ny, 9)
        probability density function
    nx: int
        number of grid points in x direction
    ny: int
        number of grid points in y direction
    
    Returns
    ---------
    f: np.array (nx, ny, 9)
        probability density function after the bounce back step
    '''
     
    # rigid lower wall 
    f[2, 1 : nx + 1, 1] = f[4, 1 : nx + 1, 0]
    f[5, 1 : nx + 1, 1] = np.roll(f[7, 1 : nx + 1, 0], 1)
    f[6, 1 : nx + 1, 1] = np.roll(f[8, 1 : nx + 1, 0], -1)
    
    # rigid upper wall
    f[4, 1 : nx + 1, ny] = f[2, 1 : nx + 1, ny + 1]
    f[7, 1 : nx + 1, ny] = np.roll(f[5, 1 : nx + 1, ny + 1], -1)
    f[8, 1 : nx + 1, ny] = np.roll(f[6, 1 : nx + 1, ny + 1], 1)

    return f    


#calulate boundary nodes X(0) and X(N+1) for periodic BC with presssure difference
def calcPeriodicBC00(_roh_N, _u_cNl, _roh_in, _roh_1, _u_c1l, _roh_out, _ltc_prestream): 

    fi_x0 = f_eq_2D(_roh_in, _u_cNl) + (_ltc_prestream[:,Xn,:] - f_eq_2D(_roh_N, _u_cNl))
    fi_xNplus1 = f_eq_2D(_roh_out, _u_c1l) + (_ltc_prestream[:,1,:] - f_eq_2D(_roh_1, _u_c1l)) 

    #return (f_eq_2D(_roh_in, _u_cNl) + (_ltc_prestream[:,Xn,:] - f_eq_2D(_roh_N, _u_cNl))), (f_eq_2D(_roh_out, _u_c1l) + (_ltc_prestream[:,1,:] - f_eq_2D(_roh_1, _u_c1l)) )
    return fi_x0, fi_xNplus1


def calcPeriodicBC0(pdf, _roh_N, _u_cNl, _roh_in, _roh_1, _u_c1l, _roh_out, _ltc_prestream, roh, u_ckl, iteration): 

    f_eq_in = f_eq_2D(_roh_in, _u_cNl)
    fi_xN_prestream = _ltc_prestream[:,Xn,:]
    fi_eq_N = f_eq_2D(_roh_N, _u_cNl)
    fi_x0 = f_eq_in + (fi_xN_prestream - fi_eq_N)

    #del f_eq_in
    #del fi_xN_prestream
    #del fi_eq_N
    #gc.collect()

    f_eq_out = f_eq_2D(_roh_out, _u_c1l)
    fi_x0_prestream = _ltc_prestream[:,1,:]
    fi_eq_1 = f_eq_2D(_roh_1, _u_c1l)
    fi_xNplus1 = f_eq_out + (fi_x0_prestream - fi_eq_1) 

    #del f_eq_out
    #del fi_x0_prestream
    #del fi_eq_1
    #gc.collect()

    return fi_x0, fi_xNplus1    


def calcPeriodicBC1(f, u, rho_in, rho_out, nx, ny):
    # Pressure Boundary conditions
    #u = velocity_calc(f, density_calc(f))
    #rho = density_calc(f)
    rho, u = updateMoments(f)
    buffer = np.ones((1, ny + 2))
    f_eq_in  = f_eq_cal(buffer * rho_in,  u[:, nx : nx + 1, :], -1, ny)
    f_eq_out = f_eq_cal(buffer * rho_out, u[:, 1 : 2, :], -1, ny)

    #f_eq = f_eq_cal(rho, velocity_calc(f, rho), nx, ny)
    f_eq = f_eq_cal(rho, u, nx, ny)

    fi_x0 = f_eq_in + (f[:, nx : nx + 1, :] - f_eq[:, nx : nx + 1, :])
    fi_xNplus1 = f_eq_out + (f[:, 1 : 2, :] - f_eq[:, 1 : 2, :])

    return fi_x0.reshape(9, ny+2), fi_xNplus1.reshape(9, ny+2)


def f_eq_cal(rho, u, nx, ny):
    '''Calulates the equilibrium probability density for a given velocity and density

    Arguments
    -----------
    nx: int
        number of grid points in x direction
    ny: int
        number of grid points in y direction
    rho: np.array (nx, ny)
        array containing the density of each grid point
    u: np.array (nx, ny, 2)
        array containing the velocity at each grid point
    
    Returns
    -----------
    f_eq: np.array (nx, ny, 9)
        equilibrium probability density
    '''
    global weights, discrete_velocities
    c_u = np.einsum('vi, ixy -> vxy', discrete_velocities, u)
    c_u_2 = c_u ** 2 # np.einsum('xyv, xyv -> xyv', c_u, c_u)
    #u_2 = np.einsum('jki, jki -> jk', u, u)
    u_2 = np.einsum('kij, kij -> ij', u, u)

    f_eq = np.zeros((9, nx + 2, ny + 2))
    for i in range(9):
        #f_eq[:,:,i] = weights[i] * rho * (1 + 3 * c_u[:,:,i] + 9 / 2 * c_u_2[:,:,i] - 3 / 2 * u_2)
        f_eq[i,:,:] = weights[i] * rho * (1 + 3 * c_u[i,:,:] + 9 / 2 * c_u_2[i,:,:] - 3 / 2 * u_2)
    return f_eq   


def compute_new_f(f, w, rho, u, ny, nx, rho_in, rho_out):
    '''Performs the collision step and applies the pressure boundaries
    
    Arguements
    -----------
    f: np.array (nx, ny, 9)
        probability density function
    w: float
        time step / time constant
    rho: np.array (nx, ny)
        array containing the denisty at each grid point
    u: np.array (nx, ny, 2)
        array containing the velocity at each grid point
    nx: int
        number of grid points in x direction
    ny: int
        number of grid points in y direction
    rho_in: float
        density at the inlet
    rho_out: float
        density at the outlet
    
    Returns
    ---------
    f: np.array (nx, ny, 9)
        probability density function after the collision step
    '''
    #
    # Collistion step
    f_eq = f_eq_cal(rho, u, nx, ny)
    f[:, 1 : nx + 1, 1 : ny + 1] = f[:, 1 : nx + 1, 1 : ny + 1] + w * (f_eq[:, 1 : nx + 1, 1 : ny + 1] - f[:, 1 : nx + 1, 1 : ny + 1])
    #
    # Pressure Boundary conditions
    
    #u = velocity_calc(f, density_calc(f))
    #rho = density_calc(f)
    rho, u = updateMoments(f)
    buffer = np.ones((1, ny + 2))
    f_eq_in  = f_eq_cal(buffer * rho_in,  u[:, nx : nx + 1, :], -1, ny)
    f_eq_out = f_eq_cal(buffer * rho_out, u[:, 1 : 2, :], -1, ny)

    #f_eq = f_eq_cal(rho, velocity_calc(f, rho), nx, ny)
    f_eq = f_eq_cal(rho, u, nx, ny)

    if False:
        print("f shape:", f.shape)
        print("f_eq shape:", f_eq.shape)
        print("f[:, nx, :].shape:", f[:, nx : nx + 1, :].shape)
        print("f_eq[:, nx, :].shape:", f_eq[:, nx : nx + 1, :].shape)
        print("f_eq_in shape:", f_eq_in.shape)    
        print("f[:, 0, :]:", f[:, 0, :].shape) 
        print("f_eq_in:", f_eq_in)    

    f[:, 0:1, :] = f_eq_in + (f[:, nx : nx + 1, :] - f_eq[:, nx : nx + 1, :])
    f[:, nx + 1:-1, :] = f_eq_out + (f[:, 1 : 2, :] - f_eq[:, 1 : 2, :])
    return f    
    

#2D Poiseuille inlet velocity u(y) for comparison with numerical result
def Poiseuille2DUy(y):
    u_poiseuille = np.zeros((2), dtype=float)
    u_y = U * (1 - ((y - D/2)/(D/2))**2)
    u_poiseuille[0] = u_y
    u_poiseuille[1] = 0

    return u_poiseuille   


def Poiseuille2DUy2(y, U, D):
    """
    Returns the velocity profile for Poiseuille flow in a pipe.
    :param y: Array of radial distances (y values).
    :param U: Maximum velocity at the center of the pipe.
    :param D: Diameter of the pipe.
    :return: Velocity profile at each y position.
    """
    R = D / 2  # Pipe radius
    u_poiseuille = U * (1 - (y / R) ** 2)
    return u_poiseuille  


def get_adjusted_velocity_y_values4Poiseuille2D(num_nodes1, D):
    # Define the spacing factor for this array as well
    delta = D / (num_nodes1 - 1)
    R = D / 2.0
    
    # Initialize the velocity_y_values array with proper non-uniform spacing
    velocity_y_values4Poiseuille2D = np.zeros(num_nodes1)
    
    # Middle points
    velocity_y_values4Poiseuille2D = np.linspace(-R, R, num_nodes1)
    
    return velocity_y_values4Poiseuille2D


def plot_multiple_frames_amplitude(list, list_frames, fig_size, axis, xlabel, ylabel, nx, ny):
    '''Plots the amplitude of multiple frames
    
    Arguements
    -----------
    list: list of arrays
        list containing the density or velocity at each simulation step
    list_frames: list of ints
        frames which are to be plotted
    fig_size: tuple of ints
        size of the figure
    axis: np.array
        x axis for plotting
    nx, ny: int
        number of grid points in x and y direction
    xlabel, ylabel: str
        labels for plotting

    Returns 
    -------
    None
    '''
    plt.figure(figsize = fig_size)
    # axis = np.hstack((axis, np.array([nx])))
    for n in range(len(list_frames)):
        u_or_rho = list[list_frames[n]][0, nx // 2, 1 : ny + 1] # u_or_rho = np.hstack((list[list_frames[n]][:,L // 4], np.array(list[list_frames[n]][0][L//4])))
        plt.plot(axis, u_or_rho, label = "t = " + str(list_frames[n]))
    plt.legend(ncol = len(list_frames) // 4, loc = 'upper right')
    plt.grid()
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)    


def density_plot(den_eq, nx, ny):
    # Plot the density profile
    fig, ax = plt.subplots(figsize = (17, 5))
    #plt.figure()
    ax.plot(den_eq[1:nx, ny // 2])
    #plt.plot(np.arange(1, nx + 1), den_eq[1 : nx + 1, ny // 2])
    #plt.grid()
    #plt.xlabel('x Achse')
    #plt.ylabel('Dichte')

In [5]:
#preliminary
#lattice for phase space; Nx+3 is due to periodic boundary conditions
#Nx is the number of divisions in the x-direction, thus there are Nx+3 points when including the extra nodes 0 and N+1 in x-direction
#lattice columns start with 0 and end with Nx+2, X(0) = X(0) and X(N+1) = X(Nx+2)

#initialise
#average velocity, cartesion x,y-directions, k is y-position, l is x-position
t = 0
roh_in_k = np.full((Yn+2), roh_in, dtype=np.float32)
roh_out_k = np.full((Yn+2), roh_out, dtype=np.float32)
#u_ckl = np.zeros((2, Yn+2, Xn+2), dtype=np.float32)
u_ckl = np.zeros((2, Xn+2, Yn+2), dtype=np.float32)
INIT_ROH = 1 #0.001
#roh = np.full((Yn+2, Xn+2), INIT_ROH, dtype=np.float32)
roh = np.full((Xn+2, Yn+2), INIT_ROH, dtype=np.float32)
roh[0,:] = roh_in_k
roh[Xn+1,:] = roh_out_k
#streamed_lattice = f_eq_slc(roh, u_ckl)
pdf = f_eq_3D(roh, u_ckl)


#roh = np.full((Ny, Nx+2), INIT_ROH, dtype=float)
#_roh_1 = roh[:,1]
#_roh_N = roh[:,Nx]

#1st prestreamed lattice
#prestream_lattice = f_eq_slc(0, roh, u_ckl)
#streamed_lattice = prestream_lattice.copy()
#printSelLtcPoints(index, "initialised prestream_lattice", prestream_lattice)

# Simulation parameters
num_nodes1 = Yn #int(D/dy) + 1  # Number of nodes in inlet/outlet
R = D / 2  # Radius of the pipe
num_nodes2 = columns_to_select


#Domain decomposition
#nx = pdf.shape(1)
#ny = pdf.shape(1)
#nlocal = nx//size
#if rank == size - 1:
#    nlocal = nx - nlocal*(size-1)
#nx1 = nlocal*rank
#nx2 = nlocal*(rank+1)
#if rank == size-1:
#    nx2 = nx
#    x = np.arange(nx1-1, nx2+1)*dx


# Enable interactive mode
n = 10  # Update the graph every 10 iterations

num_x_values = roh.shape[0]
x_all = np.linspace(0, L, num_x_values)  # Example of all possible x-values
scaled_x = x_all[columns_to_select]  # Get true x-values for selected columns

#velocity_y_values = range(num_nodes1)
#velocity_y_values4Poiseuille2D = np.linspace(-R, R, num_nodes1)

#num_nodes1 = 11
velocity_y_values = range(1, Yn+1)
print("num_nodes1:")
print(num_nodes1)
print("velocity_y_values:")
print(velocity_y_values)
velocity_y_values4Poiseuille2D = get_adjusted_velocity_y_values4Poiseuille2D(num_nodes1, D)
print("velocity_y_values4Poiseuille2D:")
print(velocity_y_values4Poiseuille2D)

x_labels = [1, 2, 3, 10, 20, 50, Xn]
min_value = 0
density_max = 1.5

iterations = []
maxRoh = []
VERBOSE2 = False

list_streaming = [u_ckl]

num_nodes1:
50
velocity_y_values:
range(1, 51)
velocity_y_values4Poiseuille2D:
[-5.00000000e-04 -4.79591837e-04 -4.59183673e-04 -4.38775510e-04
 -4.18367347e-04 -3.97959184e-04 -3.77551020e-04 -3.57142857e-04
 -3.36734694e-04 -3.16326531e-04 -2.95918367e-04 -2.75510204e-04
 -2.55102041e-04 -2.34693878e-04 -2.14285714e-04 -1.93877551e-04
 -1.73469388e-04 -1.53061224e-04 -1.32653061e-04 -1.12244898e-04
 -9.18367347e-05 -7.14285714e-05 -5.10204082e-05 -3.06122449e-05
 -1.02040816e-05  1.02040816e-05  3.06122449e-05  5.10204082e-05
  7.14285714e-05  9.18367347e-05  1.12244898e-04  1.32653061e-04
  1.53061224e-04  1.73469388e-04  1.93877551e-04  2.14285714e-04
  2.34693878e-04  2.55102041e-04  2.75510204e-04  2.95918367e-04
  3.16326531e-04  3.36734694e-04  3.57142857e-04  3.77551020e-04
  3.97959184e-04  4.18367347e-04  4.38775510e-04  4.59183673e-04
  4.79591837e-04  5.00000000e-04]


In [ ]:
VERBOSE = False
TOTAL_ITERATION = 12001
start = time.perf_counter()

while iteration < TOTAL_ITERATION:
    #1. moment update
    if iteration > 0:
        roh, u_ckl = updateMoments(pdf)
        #override the densities at 0 and N+1 using ideal gas law in isothermal ﬂuid ﬂow
        # p = cs**2 * roh 
    #enforce densities at boundary points and inlet and outlet
    #roh[:, 0] = roh_in
    #roh[:, Xn+1] = roh_out

    # Get the maximum density and its location
    max_density = np.max(roh)
    max_location = np.unravel_index(np.argmax(roh), roh.shape)
    if VERBOSE:
        if np.any(roh > 1):
            print(f"Instability detected at iteration {iteration + 1}")
        print(f"Maximum density: {max_density} at location {max_location}")


    #2. compute equilibrium
    f_eq = f_eq_3D(roh, u_ckl)

    #3. collision term
    #compact version
    #pdf = pdf * (1 - omega_nd) + omega_nd * f_eq 
    #verbose version
    #pdf = pdf - pdf*omega_nd + omega_nd*f_eq 
    #Solution version
    #f[1 : nx + 1, 1 : ny + 1, :] = f[1 : nx + 1, 1 : ny + 1, :] + w * (f_eq[1 : nx + 1, 1 : ny + 1, :] - f[1 : nx + 1, 1 : ny + 1, :])    
    #range: [:, 1 : Xn + 1, 1 : Yn + 1]
    pdf[:, 1 : Xn + 1, 1 : Yn + 1] = pdf[:, 1 : Xn + 1, 1 : Yn + 1] - pdf[:, 1 : Xn + 1, 1 : Yn + 1]*omega_nd + omega_nd*f_eq[:, 1 : Xn + 1, 1 : Yn + 1]
    #range: [:, :, 1 : Yn + 1]
    #pdf[:, :, 1 : Yn + 1] = pdf[:, :, 1 : Yn + 1] - pdf[:, :, 1 : Yn + 1]*omega_nd + omega_nd*f_eq[:, :, 1 : Yn + 1]
    #range: [:, :, 1 : Yn + 1]
    #pdf[:, :, :] = pdf[:, :, :] - pdf[:, :, :]*omega_nd + omega_nd*f_eq[:, :, :]    
    #range: [:, 2 : Xn, 1 : Yn + 1]
    #pdf[:, 2 : Xn, 1 : Yn + 1] = pdf[:, 2 : Xn, 1 : Yn + 1] - pdf[:, 2 : Xn, 1 : Yn + 1]*omega_nd + omega_nd*f_eq[:, 2 : Xn, 1 : Yn + 1]   
    #range: [:, 1 : Xn + 1, 1 : Yn + 1]
    #pdf[:, 1 : Xn + 1, 2 : Yn] = pdf[:, 1 : Xn + 1, 2 : Yn] - pdf[:, 1 : Xn + 1, 2 : Yn]*omega_nd + omega_nd*f_eq[:, 1 : Xn + 1, 2 : Yn]   

    _ltc_pre = pdf.copy()
    

    #4.1a Periodic Boundary conditions inlet/outlet with pressure difference
    #update extra node layers 0 and N+1 -> A) & B) acc. to Script: Boundary Conditions for the Lattice Boltzmann Method
    u_c1l = u_ckl[:, 1, :]
    u_cNl = u_ckl[:, Xn, :]    
    #u_ckl profiles at outlet and inlet
    _roh_k1 = roh[1, :]  
    _roh_kN = roh[Xn, :]    
    #assign inlet and outlet boundary values -> A)    
    #fi_x0, fi_xNplus1 = calcPeriodicBC0(pdf, _roh_kN, u_cNl, roh_in_k, _roh_k1, u_c1l, roh_out_k, _ltc_pre, roh, u_ckl, iteration)
    fi_x0, fi_xNplus1 = calcPeriodicBC1(pdf, u_ckl, roh_in_k, roh_out_k, Xn, Yn)


    #5. stream lattice 
    #store pre-stream boundary top and bottom values
    
    #pdf = streamLattice(pdf)
    pdf = streamLattice1(pdf)

    #4.2 Bounce-Back Top and Bottom
    #pdf = bounceBackTopBottom0(pdf, _ltc_pre, index_mapping_top, index_mapping_bottom)
    pdf = bounceBackTopBottom1(pdf, Xn, Yn)
    


    #4.1b. assign inlet boundary values -> B)
    pdf[:, 0, :] = fi_x0
    if (iteration % 100) == 0:
        if VERBOSE2:
            print("============================================================================")
            print("fi_x0: Rank: ")
            print(fi_x0) 
    pdf[:, Xn+1, :] = fi_xNplus1
    if (iteration % 100) == 0:
        if VERBOSE2:
            print("============================================================================")
            print("fi_xNplus1: Rank: ")
            print(fi_xNplus1) 

    # Update plots
    #updatePlots(iteration, pdf, u_ckl)
    last_den = roh
    list_streaming.append(u_ckl)

    t += dt
    iteration += 1
    if False:
        if (iteration % 100) == 0:
            print("\n\nindex: {0} ------------------------------------------------".format(iteration))        
            print(f"Simulation Execution -> TOTAL_ITERATION: {TOTAL_ITERATION}; t: {t}; %: {(iteration/TOTAL_ITERATION)*100.0}")

    clear_output(wait=True)  # Clears the previous output
    if (iteration % 100) == 0:
        print(f"Simulation Execution -> TOTAL_ITERATION: {TOTAL_ITERATION}; iteration: {iteration}; {((iteration/TOTAL_ITERATION)*100.0):.1f} %")

# Turn off interactive mode and display the final plot
plot_multiple_frames_amplitude(list_streaming, [0, 10, 50, 100, 200, 500, 1000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000], (17, 10), np.arange(1, Yn + 1), "y Achse", "Geschwindigkeit u$_x$", Xn, Yn)
density_plot(last_den, Xn, Yn)

profiler.stop()
print(profiler.output_text(unicode=True, color=True))

end = time.perf_counter()
print(f"Execution time: {end - start:.6f} seconds")
y = np.arange(0.5, Yn + 1, 1)
rho_in, rho_out, cs_2 = last_den[1, Yn // 2], last_den[Xn, Yn // 2], 1 / 3
dp = (rho_in - rho_out) * cs_2
print(f"rho_in: {rho_in}")
print(f"rho_out: {rho_out}")
print(f"dp: {dp}")

Simulation Execution -> TOTAL_ITERATION: 12001; iteration: 8600; 71.7 %
